In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

단어 집합을 생성하고 크기를 확인해보겠습니다. 단어 집합의 크기를 저장할 때는 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만, 패딩을 위한 0을 고려하여 +1을 해줍니다.

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
tokenizer.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [5]:
#훈련데이터 생성
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [7]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.
우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다. 이를 코드로는 다음과 같이 구할 수 있습니다.

In [8]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [10]:
#길이가 6으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

pad_sequences()는 모든 샘플에 대해서 0을 사용하여 길이를 맞춰줍니다. 
maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며, padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁니다. 
전체 훈련 데이터를 출력해봅니다.

In [15]:
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [ ]:
#마지막 단어 레이블로 지정

In [12]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [17]:
X,y

(array([[ 0,  0,  0,  0,  2],
        [ 0,  0,  0,  2,  3],
        [ 0,  0,  2,  3,  1],
        [ 0,  2,  3,  1,  4],
        [ 0,  0,  0,  0,  6],
        [ 0,  0,  0,  6,  1],
        [ 0,  0,  0,  0,  8],
        [ 0,  0,  0,  8,  1],
        [ 0,  0,  8,  1,  9],
        [ 0,  8,  1,  9, 10],
        [ 8,  1,  9, 10,  1]]),
 array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11]))

In [21]:
y = to_categorical(y, num_classes=vocab_size)

# 모델 설계

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

하이퍼파라미터인 임베딩 벡터의 차원은 10, 은닉 상태의 크기는 32입니다.
<br>다 대 일 구조의 RNN을 사용합니다. 
<br>전결합층(Fully Connected Layer)을 출력층으로 단어 집합 크기만큼의 뉴런을 배치하여 모델을 설계합니다.
<br>해당 모델은 마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다. 
<br>다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용하여 200 에포크를 수행합니다

In [22]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4671 - accuracy: 0.2727 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4481 - accuracy: 0.2727 - 4ms/epoch - 4ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4293 - accuracy: 0.2727 - 5ms/epoch - 5ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4106 - accuracy: 0.1818 - 4ms/epoch - 4ms/step
Epoch 5/200
1/1 - 0s - loss: 2.3919 - accuracy: 0.2727 - 5ms/epoch - 5ms/step
Epoch 6/200
1/1 - 0s - loss: 2.3731 - accuracy: 0.2727 - 7ms/epoch - 7ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3542 - accuracy: 0.2727 - 6ms/epoch - 6ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3350 - accuracy: 0.4545 - 6ms/epoch - 6ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3154 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 10/200
1/1 - 0s - loss: 2.2954 - accuracy: 0.4545 - 4ms/epoch - 4ms/step
Epoch 11/200
1/1 - 0s - loss: 2.2749 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 12/200
1/1 - 0s - loss: 2.2538 - accuracy: 0.4545 - 4ms/epoch - 4ms/step
Epoch 13/200
1/1 - 0s - loss: 2.2320 - accuracy: 0.4545 - 5ms/e

Epoch 105/200
1/1 - 0s - loss: 0.6348 - accuracy: 0.8182 - 4ms/epoch - 4ms/step
Epoch 106/200
1/1 - 0s - loss: 0.6247 - accuracy: 0.8182 - 4ms/epoch - 4ms/step
Epoch 107/200
1/1 - 0s - loss: 0.6148 - accuracy: 0.8182 - 3ms/epoch - 3ms/step
Epoch 108/200
1/1 - 0s - loss: 0.6051 - accuracy: 0.8182 - 6ms/epoch - 6ms/step
Epoch 109/200
1/1 - 0s - loss: 0.5955 - accuracy: 0.8182 - 4ms/epoch - 4ms/step
Epoch 110/200
1/1 - 0s - loss: 0.5860 - accuracy: 0.8182 - 5ms/epoch - 5ms/step
Epoch 111/200
1/1 - 0s - loss: 0.5768 - accuracy: 0.8182 - 4ms/epoch - 4ms/step
Epoch 112/200
1/1 - 0s - loss: 0.5676 - accuracy: 0.8182 - 4ms/epoch - 4ms/step
Epoch 113/200
1/1 - 0s - loss: 0.5587 - accuracy: 0.8182 - 5ms/epoch - 5ms/step
Epoch 114/200
1/1 - 0s - loss: 0.5498 - accuracy: 0.9091 - 3ms/epoch - 3ms/step
Epoch 115/200
1/1 - 0s - loss: 0.5412 - accuracy: 0.9091 - 4ms/epoch - 4ms/step
Epoch 116/200
1/1 - 0s - loss: 0.5326 - accuracy: 0.9091 - 4ms/epoch - 4ms/step
Epoch 117/200
1/1 - 0s - loss: 0.5242 - 

# test

In [23]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [28]:
print(sentence_generation(model, tokenizer, '경마장에', 2))

경마장에 있는 말이


In [25]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [26]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다
